In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f


In [2]:
#SparkSession creation
spark = SparkSession.builder.appName("MovieLensRating").getOrCreate()

In [33]:
df_data = (
    spark.read.csv(
        path="D:/New folder (3)/movielens/u_data.csv",
        sep= ",",
        quote='"',
        header=True,
        schema="userId INT, movieID INT, rating DOUBLE, timestamp INT",
     )
    #converting timestamp format to regular one from unixtime
     .withColumn("timestamp",f.to_timestamp(f.from_unixtime("timestamp")))
)

df_user = spark.read.csv(
             path="D:/New folder (3)/movielens/u_user.csv",
             sep=",",
             quote='"',
             header=True,
             schema="userId INT, age STRING, gender STRING,occupation STRING, zipcode INT"
        )



In [34]:
df_user.show()
df_data.show()

+------+---+------+-------------+-------+
|userId|age|gender|   occupation|zipcode|
+------+---+------+-------------+-------+
|     1| 24|     M|   technician|  85711|
|     2| 53|     F|        other|  94043|
|     3| 23|     M|       writer|  32067|
|     4| 24|     M|   technician|  43537|
|     5| 33|     F|        other|  15213|
|     6| 42|     M|    executive|  98101|
|     7| 57|     M|administrator|  91344|
|     8| 36|     M|administrator|   5201|
|     9| 29|     M|      student|   1002|
|    10| 53|     M|       lawyer|  90703|
|    11| 39|     F|        other|  30329|
|    12| 28|     F|        other|   6405|
|    13| 47|     M|     educator|  29206|
|    14| 45|     M|    scientist|  55106|
|    15| 49|     F|     educator|  97301|
|    16| 21|     M|entertainment|  10309|
|    17| 30|     M|   programmer|   6355|
|    18| 35|     F|        other|  37212|
|    19| 40|     M|    librarian|   2138|
|    20| 42|     F|    homemaker|  95660|
+------+---+------+-------------+-

In [35]:
#Joining opinion table with rating on two unique ids i.e "movieId" and "userId": inner join
opinions_ext = (df_user.join(df_data,["userId"],"inner")
#             .select("user_Id","movieID","title","tag","rating","timestamp_tag","timestamp")
        )
opinions_ext.show(10,False)

+------+---+------+----------+-------+-------+------+-------------------+
|userId|age|gender|occupation|zipcode|movieID|rating|timestamp          |
+------+---+------+----------+-------+-------+------+-------------------+
|196   |49 |M     |writer    |55105  |242    |3.0   |1997-12-04 20:55:49|
|186   |39 |F     |executive |0      |302    |3.0   |1998-04-05 00:22:22|
|22    |25 |M     |writer    |40206  |377    |1.0   |1997-11-07 12:18:36|
|244   |28 |M     |technician|80525  |51     |2.0   |1997-11-27 10:02:03|
|166   |47 |M     |educator  |55113  |346    |1.0   |1998-02-02 10:33:16|
|298   |44 |M     |executive |1581   |474    |4.0   |1998-01-07 19:20:06|
|115   |31 |M     |engineer  |17110  |265    |2.0   |1997-12-03 22:51:28|
|253   |26 |F     |librarian |22903  |465    |5.0   |1998-04-03 23:34:27|
|305   |23 |M     |programmer|94086  |451    |3.0   |1998-02-01 14:20:17|
|6     |42 |M     |executive |98101  |86     |3.0   |1998-01-01 02:16:53|
+------+---+------+----------+-------+

In [47]:
opinions_ext.groupBy("movieID","occupation").agg(
    f.count("*").alias("Total No. of Rating"),
    f.min("rating").alias("Min Rating"),
    f.max("rating").alias("Max Rating"),
    f.avg("rating").alias("Avg Rating"),
    f.min("timestamp").alias("Earliest"),
    f.max("timestamp").alias("Latest"),
).show(21,False)

+-------+-------------+-------------------+----------+----------+------------------+-------------------+-------------------+
|movieID|occupation   |Total No. of Rating|Min Rating|Max Rating|Avg Rating        |Earliest           |Latest             |
+-------+-------------+-------------------+----------+----------+------------------+-------------------+-------------------+
|785    |student      |11                 |2.0       |5.0       |3.4545454545454546|1997-09-20 09:36:46|1998-03-31 06:55:51|
|603    |marketing    |7                  |3.0       |5.0       |4.714285714285714 |1997-09-29 18:00:27|1998-04-01 18:59:31|
|289    |educator     |31                 |1.0       |5.0       |3.0               |1997-10-07 22:23:11|1998-04-07 08:46:03|
|134    |doctor       |1                  |4.0       |4.0       |4.0               |1997-10-30 11:18:31|1997-10-30 11:18:31|
|756    |entertainment|3                  |2.0       |3.0       |2.3333333333333335|1998-01-06 06:31:49|1998-02-02 23:19:42|


In [45]:
opinions_ext.groupby("userId","movieID","occupation").agg(
#     f.collect_set("movieID").alias("MOvieIDS"),
    f.count("*").alias("Ratings Given"),
    f.avg("rating").alias("Avg Rating Given"),
    f.min("rating").alias("Min Rating given"),
    f.max("rating").alias("Max Rating Given"),
).sort("Ratings Given",ascending=False).show(20)

+------+-------+-------------+-------------+----------------+----------------+----------------+
|userId|movieID|   occupation|Ratings Given|Avg Rating Given|Min Rating given|Max Rating Given|
+------+-------+-------------+-------------+----------------+----------------+----------------+
|    95|   1217|administrator|            1|             3.0|             3.0|             3.0|
|    44|    449|   technician|            1|             5.0|             5.0|             5.0|
|     7|    200|administrator|            1|             5.0|             5.0|             5.0|
|   292|      9|   programmer|            1|             4.0|             4.0|             4.0|
|     1|     47|   technician|            1|             4.0|             4.0|             4.0|
|   311|    559|   technician|            1|             2.0|             2.0|             2.0|
|   116|    751|   healthcare|            1|             3.0|             3.0|             3.0|
|    79|    100|administrator|          